JDC: I've been printing the work accumulated over the first 5000 steps with the default K_RMSD compared with K_RMSD=0 and only changing one set of lambda_* parameters at a time.
It appears that changing the bond lambdas causes the biggest difference in work accumulation between with and without restraints

Script: /home/chodera/equilibrated_restraint_experiments/281/test_neq.py

In [1]:
import openmm
from openmm import unit
from openmmtools.integrators import PeriodicNonequilibriumIntegrator

# prefix = '/data/chodera/zhangi/perses_benchmark/neq/15/281/equilibrated/281/'
prefix = '/home/chodera/equilibrated_restraint_experiments/281/'

def read(filename):
    print(f'Reading {filename}...')
    with open(filename, 'r') as infile:
        return openmm.XmlSerializer.deserialize(infile.read())
    
def create_integrator(param_to_change=None):
    nsteps_eq = 1
    nsteps_neq = 250000 # 1 ns
    neq_splitting='V R H O R V'
    timestep = 4.0 * unit.femtosecond
    platform_name = 'CUDA'
    temperature = 300.0 * unit.kelvin

    # Define lambda functions
    x = 'lambda'

    ALCHEMICAL_FUNCTIONS = {
    'lambda_rest_bonds': "1",
    'lambda_rest_angles': "1",
    'lambda_rest_torsions':"1",
    'lambda_rest_electrostatics': "1",
    'lambda_rest_electrostatics_exceptions': "1",
    'lambda_rest_sterics':"1",
    'lambda_rest_sterics_exceptions': "1", 
                             'lambda_alchemical_bonds_old': f'1 - {x}',
                             'lambda_alchemical_bonds_new': x,
                             'lambda_alchemical_angles_old': f'1 - {x}',
                             'lambda_alchemical_angles_new': x,
                             'lambda_alchemical_torsions_old': f'1 - {x}',
                             'lambda_alchemical_torsions_new': x,
                             'lambda_alchemical_electrostatics_old': f'1 - {x}',
                             'lambda_alchemical_electrostatics_new': x,
                             'lambda_alchemical_electrostatics_exceptions_old': f'1 - {x}',
                             'lambda_alchemical_electrostatics_exceptions_new': x,
                             'lambda_alchemical_electrostatics_reciprocal': x,
                             'lambda_alchemical_sterics_old': f'1 - {x}',
                             'lambda_alchemical_sterics_new': x,
                             'lambda_alchemical_sterics_exceptions_old': f'1 - {x}',
                             'lambda_alchemical_sterics_exceptions_new': x
                             }
    
    if param_to_change is not None:
        for k, v in ALCHEMICAL_FUNCTIONS.items():
            if 'old' in k:
                if param_to_change in k:
                    pass
                else:
                    ALCHEMICAL_FUNCTIONS[k] = '1.0'
            elif 'new' in k or 'reciprocal' in k:
                if param_to_change in k:
                    pass
                else:
                    ALCHEMICAL_FUNCTIONS[k] = '0.0'
                
    return PeriodicNonequilibriumIntegrator(ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)
    
    
def test(param_to_change=None):
    
    system = read(f'{prefix}281_apo_system.xml')
    state = read(f'{prefix}281_apo_state.xml')
#     integrator = read(f'{prefix}integrator.xml')
    integrator = create_integrator(param_to_change)

    forces = { force.getName() : force for (index, force) in enumerate(system.getForces()) }
    cvforce = forces['CustomCVForce']

    platform = openmm.Platform.getPlatformByName('CUDA')
    platform.setPropertyDefaultValue('Precision', 'mixed')
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
    
    print("************K_RMSD != 0************")
    print('Creating context...')

    context = openmm.Context(system, integrator, platform)
    context.setState(state)

    print('Integrating...')
    niterations = 5
    nsteps_per_iteration = 1000
    for iteration in range(niterations):
            
        step = integrator.getGlobalVariableByName('step')
        lambda_ = integrator.getGlobalVariableByName('lambda')
        protocol_work = integrator.getGlobalVariableByName('protocol_work')
        time = context.getTime()
        cvs = cvforce.getCollectiveVariableValues(context)
        rmsd = cvs[0]
        print(f'step {step:10.0f} : time {time/unit.nanoseconds:10.3f} ns | lambda {lambda_:16.12f} | protocol_work = {protocol_work:12.3f} | RMSD = {rmsd:12.5f} nm')
        integrator.step(nsteps_per_iteration)

    del context, integrator
    
    print("************K_RMSD == 0************")
    print('Creating context...')

#     integrator = read(f'{prefix}integrator.xml')
    integrator = create_integrator(param_to_change)
    
    context = openmm.Context(system, integrator, platform)
    context.setState(state)
    
    print(f'Setting K_RMSD to 0')
    context.setParameter('K_RMSD', 0.0)

    print('Integrating...')
    niterations = 5
    nsteps_per_iteration = 1000
    for iteration in range(niterations):
     
        step = integrator.getGlobalVariableByName('step')
        lambda_ = integrator.getGlobalVariableByName('lambda')
        protocol_work = integrator.getGlobalVariableByName('protocol_work')
        time = context.getTime()
        cvs = cvforce.getCollectiveVariableValues(context)
        rmsd = cvs[0]
        print(f'step {step:10.0f} : time {time/unit.nanoseconds:10.3f} ns | lambda {lambda_:16.12f} | protocol_work = {protocol_work:12.3f} | RMSD = {rmsd:12.5f} nm')
        integrator.step(nsteps_per_iteration)
    
    del context, integrator



## Run neq

In [2]:
test()

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        1.169 | RMSD =      0.07026 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        3.962 | RMSD =      0.08530 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        7.148 | RMSD =      0.09075 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =       10.329 | RMSD =      0.09666 nm
************K_RMSD == 0************
Creating context...
Setting K_RMSD to 0
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work

## Run neq with lambda_bonds = 0

In [3]:
test('bonds')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.135 | RMSD =      0.06069 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.349 | RMSD =      0.07113 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.437 | RMSD =      0.08559 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =        0.512 | RMSD =      0.08539 nm
************K_RMSD == 0************
Creating context...
Setting K_RMSD to 0
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work

## Run neq with lambda_angles = 0

In [4]:
test('angles')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.138 | RMSD =      0.06368 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.253 | RMSD =      0.07802 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.334 | RMSD =      0.08228 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =        0.418 | RMSD =      0.09028 nm
************K_RMSD == 0************
Creating context...
Setting K_RMSD to 0
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work

## Run neq with lambda_torsions = 0

In [5]:
test('torsions')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.134 | RMSD =      0.06054 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.215 | RMSD =      0.07148 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.288 | RMSD =      0.08279 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =        0.416 | RMSD =      0.08066 nm
************K_RMSD == 0************
Creating context...
Setting K_RMSD to 0
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work

## Run neq with lambda_electrostatics = 0

In [6]:
test('electrostatics')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        3.935 | RMSD =      0.06084 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        7.945 | RMSD =      0.07860 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =       11.915 | RMSD =      0.08397 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =       15.765 | RMSD =      0.08775 nm
************K_RMSD == 0************
Creating context...
Setting K_RMSD to 0
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work

## Run neq with lambda_sterics = 0

In [7]:
test('sterics')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =       -2.646 | RMSD =      0.05977 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =       -3.733 | RMSD =      0.07554 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =       -4.553 | RMSD =      0.08425 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =       -5.226 | RMSD =      0.08870 nm
************K_RMSD == 0************
Creating context...
Setting K_RMSD to 0
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work

# Old code

In [ ]:
#         if lambda_param is not None: # Only allow one set of lambdas to be changing
#             print(f"Changing only one lambda: {lambda_param}")
#             for k, v in context.getParameters().items():
                
#                 if 'old' in k:
#                     if lambda_param in k:
#                         pass
#                     else:
#                         context.setParameter(k, 1.0)
#                 elif 'new' in k or 'reciprocal' in k:
#                     if lambda_param in k:
#                         pass
#                     else:
#                         context.setParameter(k, 0)